# Load our dataset using huggingface's load_dataset

In [41]:
from nlp import load_dataset
#dataset = load_dataset('csv', data_files={'train':'nyt_train.csv','test':'nyt_val.csv'})
dataset = load_dataset('csv', data_files={'train':'nyt_train.csv','validation':'nyt_val.csv'},delimiter='~')
print(dataset.keys())
print("Size of train dataset: ", dataset['train'].shape)
print("Size of Validation dataset: ", dataset['validation'].shape)
## Look at Sample Examples
print(dataset['train'][0].keys())
print(" Example of text: ", dataset['train'][0]['maintext'])
print(" Example of Summary: ", dataset['train'][0]['title'])
## Estimate Average Length of Text and Summary
tiny_dataset = dataset['train'].select(list(range(0, 100)))
text_len = []
summary_len=[]
for i in range(len(tiny_dataset)):
    example = tiny_dataset[i]
    text_example = example['maintext']
    text_example = text_example.replace('\n','')
    text_words = text_example.split()
    text_len.append(len(text_words))
    summary_example = example['title']
    summary_example = summary_example.replace('\n','')
    summary_words = summary_example.split()
    summary_len.append(len(summary_words))

import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(text_len)
plt.show()

Using custom data configuration default


0 tables [00:00, ? tables/s]

ArrowInvalid: CSV parse error: Expected 3 columns, got 1: Mr. Northam faced increased pressure to step down after he admitted that he had darkened his fac ...